In [ ]:
import sqlite3
import pandas as pd

In [ ]:
from adler.objectdata.objectdata_utilities import get_data_table

In [ ]:
db_fname = "mpc_obs_sbn_testing_database.sqlite"

In [ ]:
# Use the SSSC Prompt Products Database Bandaid
# This is the B612 release with additional processing, including orbit derived information from JPL Horizons - see SSSC Slack
input_sql_filename = "/Users/seanobrien/Documents/Adler/rubin.sqlite"

# # Run the additional preprocessing the first time to convert times from UTC -> TAI (creates mjd_tai column), adjust band names etc
# from adler.objectdata.objectdata_utilities import mpc_file_preprocessing
# mpc_file_preprocessing(sql_filename=input_sql_filename)

In [ ]:
conn = sqlite3.connect(db_fname)
cursor = conn.cursor()

In [ ]:
ssoid_list = ("1981 US22", "2002 SQ71", "2025 MS22", "2025 MX40", "2025 MX40Fake")

test_id = "2025 MX40"
fake_id = "2025 MX40Fake"

In [ ]:
obs_sbn_query = f"""SELECT * FROM obs_sbn WHERE provid in {ssoid_list}"""
obs_sbn_table = get_data_table(obs_sbn_query, service=None, sql_filename=input_sql_filename)

In [ ]:
mpc_orbits_query = f"""SELECT * FROM mpc_orbits WHERE fullDesignation in {ssoid_list}"""
mpc_orbits_table = get_data_table(mpc_orbits_query, service=None, sql_filename=input_sql_filename)

In [ ]:
_df = obs_sbn_table.copy()[obs_sbn_table["provid"] == test_id]
_df["provid"] = fake_id
obs_sbn_table = pd.concat([obs_sbn_table, _df], axis=0).reset_index(drop=True)
_df

In [ ]:
# add an outburst
t0 = 60796
t1 = 60800
mag_shift = -1.5

mask = (
    (obs_sbn_table["provid"] == fake_id) & (obs_sbn_table["mjd_tai"] > t0) & (obs_sbn_table["mjd_tai"] < t1)
)

obs_sbn_table.loc[mask, "mag"] += mag_shift

In [ ]:
obs_sbn_table.to_sql("obs_sbn", con=conn, if_exists="replace", index=False)

mpc_orbits_table.to_sql("mpc_orbits", con=conn, if_exists="replace", index=False)

_mpcorb_table = mpc_orbits_table.copy()[mpc_orbits_table["fullDesignation"] == test_id]
_mpcorb_table["fullDesignation"] = fake_id
_mpcorb_table.to_sql("mpc_orbits", con=conn, if_exists="append", index=False)